# الرحمن (Rahman) Scraper and CSV Updater

This notebook:
1. Scrapes all occurrences of "الرحمن" (Rahman) from corpus.quran.com
2. Processes the data to get frequency per verse and word locations
3. Adds a new column "rahman_frequency" to the existing Quran CSV
4. Adds a new column "rahman_word_locations" with the list of word positions

**Data source:** https://corpus.quran.com/search.jsp?q=%D8%B1%D8%AD%D9%85%D8%A7%D9%86

## ⚠️ IMPORTANT: Internet Required
This notebook requires internet access to scrape data from corpus.quran.com.

**Run this notebook in:**
- ✅ Kaggle (has internet access)
- ✅ Google Colab (has internet access)
- ✅ Your local Jupyter (if you have internet)
- ❌ Claude.ai Computer Use (network disabled)

## Features:
- Rejects duplicates during scraping (not after)
- Validates zero duplicates
- Complete error handling
- Progress tracking
- Follows same pattern as your Allah scraper

## Step 1: Install and Import Libraries

In [1]:
!pip install requests beautifulsoup4 pandas --break-system-packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from IPython.display import display, HTML

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 2: Define Scraping Functions

In [3]:
def scrape_rahman_page(page_num, verbose=True):
    """
    Scrape a single page of Rahman results from the Quranic corpus.
    
    Returns all results found on the page (duplicates handled by caller).
    """
    url = f"https://corpus.quran.com/search.jsp?q=%D8%B1%D8%AD%D9%85%D8%A7%D9%86&page={page_num}"
    
    if verbose:
        print(f"Scraping page {page_num}...", end=" ")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        if verbose:
            print("✓")
    except requests.RequestException as e:
        if verbose:
            print(f"✗ Error: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    results = []
    rows = soup.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        if len(cells) >= 3:
            # Extract location (e.g., "1:1:3")
            location_link = cells[0].find('a')
            if location_link:
                location = location_link.text.strip()
                
                # Parse location into chapter:verse:word_position
                location_parts = location.split(':')
                if len(location_parts) == 3:
                    chapter = int(location_parts[0])
                    verse = int(location_parts[1])
                    word_position = int(location_parts[2])
                    
                    # Extract transliteration
                    transliteration = cells[1].text.strip() if len(cells) > 1 else ""
                    
                    # Extract translation
                    translation = cells[2].text.strip() if len(cells) > 2 else ""
                    
                    # Extract Arabic verse text
                    arabic_verse = ""
                    if len(cells) > 3:
                        arabic_div = cells[3].find('div', class_='arabic')
                        if arabic_div:
                            arabic_verse = arabic_div.text.strip()
                    
                    results.append({
                        'location': location,
                        'chapter': chapter,
                        'verse': verse,
                        'word_position': word_position,
                        'transliteration': transliteration,
                        'translation': translation,
                        'arabic_verse': arabic_verse
                    })
    
    return results


def scrape_all_rahman_pages(max_pages=20, delay=1.0):
    """
    Scrape all pages of Rahman occurrences, rejecting duplicates immediately.
    
    Returns:
    - unique_results: List of unique results
    - rejected_count: Number of duplicates rejected
    """
    print("=" * 70)
    print("Starting Rahman Scraping")
    print("=" * 70)
    
    seen_locations = set()
    unique_results = []
    rejected_count = 0
    
    for page_num in range(0, max_pages):
        page_results = scrape_rahman_page(page_num)
        
        # If page returned no results, we've reached the end
        if not page_results:
            print(f"\n✓ Reached end of results at page {page_num}")
            break
        
        # Process each result, rejecting duplicates
        for result in page_results:
            location = result['location']
            if location not in seen_locations:
                seen_locations.add(location)
                unique_results.append(result)
            else:
                rejected_count += 1
        
        # Respectful delay between requests
        time.sleep(delay)
    
    print("\n" + "=" * 70)
    print(f"Scraping Complete!")
    print(f"  Total unique results: {len(unique_results)}")
    print(f"  Duplicates rejected:  {rejected_count}")
    print("=" * 70)
    
    return unique_results, rejected_count


print("✓ Scraping functions defined")

✓ Scraping functions defined


## Step 3: Scrape All Rahman Data

In [4]:
# Scrape all Rahman occurrences
rahman_results, rejected_count = scrape_all_rahman_pages(max_pages=20, delay=1.0)

Starting Rahman Scraping
Scraping page 0... ✓

✓ Reached end of results at page 0

Scraping Complete!
  Total unique results: 0
  Duplicates rejected:  0


## Step 4: Create DataFrame and Validate

In [5]:
# Create DataFrame
df_rahman = pd.DataFrame(rahman_results)

print(f"\nDataFrame created with {len(df_rahman)} rows")
print(f"Columns in scraped data: {list(df_rahman.columns)}")

# Display first few rows to see what we got
print("\nFirst 10 results:")
display(df_rahman.head(10))

print("\n" + "=" * 70)
print("VALIDATION: Checking for duplicates")
print("=" * 70)

# Check for duplicate locations
duplicate_locations = df_rahman[df_rahman.duplicated(subset=['location'], keep=False)]

if len(duplicate_locations) == 0:
    print("✓ PASS: Zero duplicates found in the dataset")
else:
    print(f"✗ FAIL: {len(duplicate_locations)} duplicate entries found")
    print("\nDuplicate locations:")
    display(duplicate_locations[['location', 'chapter', 'verse', 'word_position']].sort_values('location'))

print("\nLast 10 results:")
display(df_rahman.tail(10))


DataFrame created with 0 rows
Columns in scraped data: []

First 10 results:


""



VALIDATION: Checking for duplicates
✓ PASS: Zero duplicates found in the dataset

Last 10 results:


""


## Step 5: Process Data to Get Frequency Per Verse

In [6]:
# Define Quran structure (number of verses per surah)
verses_per_surah = [
    7, 286, 200, 176, 120, 165, 206, 75, 129, 109,
    123, 111, 43, 52, 99, 128, 111, 110, 98, 135,
    112, 78, 118, 64, 77, 227, 93, 88, 69, 60,
    34, 30, 73, 54, 45, 83, 182, 88, 75, 85,
    54, 53, 89, 59, 37, 35, 38, 29, 18, 45,
    60, 49, 62, 55, 78, 96, 29, 22, 24, 13,
    14, 11, 11, 18, 12, 12, 30, 52, 52, 44,
    28, 28, 20, 56, 40, 31, 50, 40, 46, 42,
    29, 19, 36, 25, 22, 17, 19, 26, 30, 20,
    15, 21, 11, 8, 8, 19, 5, 8, 8, 11,
    11, 8, 3, 9, 5, 4, 7, 3, 6, 3,
    5, 4, 5, 6
]

print(f"Total surahs: {len(verses_per_surah)}")
print(f"Total verses in Quran: {sum(verses_per_surah)}")

# Create a complete list of all verses in the Quran
all_verses = []
for surah_num, num_verses in enumerate(verses_per_surah, start=1):
    for verse_num in range(1, num_verses + 1):
        all_verses.append({
            'surah_no': surah_num,
            'ayah_no': verse_num,
            'rahman_frequency': 0,
            'rahman_word_locations': []
        })

df_all_verses = pd.DataFrame(all_verses)
print(f"\nCreated complete verse structure with {len(df_all_verses)} verses")

# Check if we have any Rahman data
if len(df_rahman) == 0:
    print("\n⚠️ WARNING: No Rahman data was scraped. Please check the scraping section.")
    print("Creating empty Rahman columns...")
    df_rahman_complete = df_all_verses
else:
    # Make sure 'chapter' column exists (it should from scraping)
    if 'chapter' not in df_rahman.columns:
        print("\n✗ ERROR: 'chapter' column not found in scraped data")
        print(f"Available columns: {list(df_rahman.columns)}")
        raise ValueError("Scraping did not produce expected columns")
    
    # Group Rahman occurrences by verse and aggregate
    rahman_by_verse = df_rahman.groupby(['chapter', 'verse']).agg({
        'word_position': lambda x: sorted(list(x))  # List of word positions
    }).reset_index()
    
    rahman_by_verse.columns = ['surah_no', 'ayah_no', 'rahman_word_locations']
    rahman_by_verse['rahman_frequency'] = rahman_by_verse['rahman_word_locations'].apply(len)
    
    print(f"\nGrouped Rahman data by verse: {len(rahman_by_verse)} verses contain Rahman")
    
    # Merge with complete verse list
    df_rahman_complete = df_all_verses.merge(
        rahman_by_verse,
        on=['surah_no', 'ayah_no'],
        how='left',
        suffixes=('', '_rahman')
    )
    
    # Update frequency and locations for verses that have Rahman
    df_rahman_complete['rahman_frequency'] = df_rahman_complete['rahman_frequency_rahman'].fillna(
        df_rahman_complete['rahman_frequency']
    ).astype(int)
    
    df_rahman_complete['rahman_word_locations'] = df_rahman_complete.apply(
        lambda row: row['rahman_word_locations_rahman'] if pd.notna(row['rahman_frequency_rahman']) else [],
        axis=1
    )
    
    # Keep only necessary columns
    df_rahman_complete = df_rahman_complete[['surah_no', 'ayah_no', 'rahman_frequency', 'rahman_word_locations']]
    
    print("\nProcessed Rahman frequency data:")
    print(f"  Total verses: {len(df_rahman_complete)}")
    print(f"  Verses with Rahman: {(df_rahman_complete['rahman_frequency'] > 0).sum()}")
    print(f"  Verses without Rahman: {(df_rahman_complete['rahman_frequency'] == 0).sum()}")
    print(f"  Total Rahman occurrences: {df_rahman_complete['rahman_frequency'].sum()}")
    
    # Display sample verses with Rahman
    print("\nSample verses with Rahman:")
    display(df_rahman_complete[df_rahman_complete['rahman_frequency'] > 0].head(20))

Total surahs: 114
Total verses in Quran: 6236

Created complete verse structure with 6236 verses

⚠️ WARNING: No Rahman data was scraped. Please check the scraping section.
Creating empty Rahman columns...


## Step 6: Load Existing CSV and Add Rahman Columns

In [7]:
# Load the existing Quran CSV
# ⚠️ IMPORTANT: Update this path based on your environment

# For Kaggle, if you uploaded the CSV as a dataset:
# existing_csv_path = '/kaggle/input/your-dataset-name/quran_ayahs_cleaned__3___1_.csv'

# For local Jupyter or Colab:
existing_csv_path = '/kaggle/input/datasets/axha241419/proper-noun-word-allah-extracted/quran_ayahs_cleaned (3).csv'  # Update this path!

print(f"Loading existing CSV from: {existing_csv_path}")
df_quran = pd.read_csv(existing_csv_path)

print(f"\nLoaded {len(df_quran)} rows")
print(f"Columns: {list(df_quran.columns)}")

# Display first few rows
print("\nFirst 5 rows of existing data:")
display(df_quran.head())

Loading existing CSV from: /kaggle/input/datasets/axha241419/proper-noun-word-allah-extracted/quran_ayahs_cleaned (3).csv

Loaded 6236 rows
Columns: ['serial_no', 'surah_no', 'ayah_no', 'ayah', 'frequency_proper_noun', 'label', 'length', 'tokens', 'word_count', 'word_locations_reordered']

First 5 rows of existing data:


,serial_no,surah_no,ayah_no,ayah,frequency_proper_noun,label,length,tokens,word_count,word_locations_reordered
0,1,1,1,بِسۡمِ ٱللَّهِ ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ,1,NaN,38,بِسۡمِ | ٱللَّهِ | ٱلرَّحۡمَٰنِ | ٱلرَّحِيمِ,4,[2]
1,2,1,2,ٱلۡحَمۡدُ لِلَّهِ رَبِّ ٱلۡعَٰلَمِينَ,1,NaN,37,ٱلۡحَمۡدُ | لِلَّهِ | رَبِّ | ٱلۡعَٰلَمِينَ,4,[2]
2,3,1,3,ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ,0,NaN,23,ٱلرَّحۡمَٰنِ | ٱلرَّحِيمِ,2,[]
3,4,1,4,مَٰلِكِ يَوۡمِ ٱلدِّينِ,0,NaN,23,مَٰلِكِ | يَوۡمِ | ٱلدِّينِ,3,[]
4,5,1,5,إِيَّاكَ نَعۡبُدُ وَإِيَّاكَ نَسۡتَعِينُ,0,NaN,40,إِيَّاكَ | نَعۡبُدُ | وَإِيَّاكَ | نَسۡتَعِينُ,4,[]


In [8]:
# Merge Rahman data with existing CSV
print("Merging Rahman data with existing CSV...")

df_updated = df_quran.merge(
    df_rahman_complete,
    on=['surah_no', 'ayah_no'],
    how='left'
)

# Fill any missing values with 0 frequency and empty list
df_updated['rahman_frequency'] = df_updated['rahman_frequency'].fillna(0).astype(int)
df_updated['rahman_word_locations'] = df_updated['rahman_word_locations'].apply(
    lambda x: x if isinstance(x, list) else []
)

print(f"\n✓ Merge complete!")
print(f"  Total rows: {len(df_updated)}")
print(f"  New columns added: rahman_frequency, rahman_word_locations")

# Verify the merge
print("\nVerification:")
print(f"  Verses with Rahman: {(df_updated['rahman_frequency'] > 0).sum()}")
print(f"  Total Rahman occurrences: {df_updated['rahman_frequency'].sum()}")

# Display sample verses with Rahman
print("\nSample verses with Rahman in updated CSV:")
display(df_updated[df_updated['rahman_frequency'] > 0][[
    'serial_no', 'surah_no', 'ayah_no', 'ayah', 
    'rahman_frequency', 'rahman_word_locations'
]].head(20))

Merging Rahman data with existing CSV...

✓ Merge complete!
  Total rows: 6236
  New columns added: rahman_frequency, rahman_word_locations

Verification:
  Verses with Rahman: 0
  Total Rahman occurrences: 0

Sample verses with Rahman in updated CSV:


,serial_no,surah_no,ayah_no,ayah,rahman_frequency,rahman_word_locations


## Step 7: Save Updated CSV

In [9]:
# Save the updated CSV
# Output will be saved in the current working directory
output_path = 'quran_ayahs_with_rahman.csv'

print(f"Saving updated CSV to: {output_path}")
df_updated.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n" + "=" * 70)
print("✓ SUCCESS! CSV updated and saved")
print("=" * 70)
print(f"\nOutput file: {output_path}")
print(f"Total rows: {len(df_updated)}")
print(f"Total columns: {len(df_updated.columns)}")
print(f"\nColumns in final CSV:")
for i, col in enumerate(df_updated.columns, 1):
    print(f"  {i}. {col}")

print("\n💾 You can now download this file from your environment!")

Saving updated CSV to: quran_ayahs_with_rahman.csv

✓ SUCCESS! CSV updated and saved

Output file: quran_ayahs_with_rahman.csv
Total rows: 6236
Total columns: 12

Columns in final CSV:
  1. serial_no
  2. surah_no
  3. ayah_no
  4. ayah
  5. frequency_proper_noun
  6. label
  7. length
  8. tokens
  9. word_count
  10. word_locations_reordered
  11. rahman_frequency
  12. rahman_word_locations

💾 You can now download this file from your environment!


## Step 8: Final Statistics and Summary

In [10]:
print("=" * 70)
print("FINAL STATISTICS")
print("=" * 70)

print(f"\nDataset Summary:")
print(f"  Total verses in Quran: {len(df_updated)}")
print(f"  Verses containing Rahman: {(df_updated['rahman_frequency'] > 0).sum()}")
print(f"  Verses without Rahman: {(df_updated['rahman_frequency'] == 0).sum()}")
print(f"  Total Rahman occurrences: {df_updated['rahman_frequency'].sum()}")

print(f"\nRahman Frequency Distribution:")
freq_dist = df_updated['rahman_frequency'].value_counts().sort_index()
for freq, count in freq_dist.items():
    if freq > 0:
        print(f"  {freq} occurrence(s): {count} verses")

# Find verses with multiple occurrences
multiple_rahman = df_updated[df_updated['rahman_frequency'] > 1]
if len(multiple_rahman) > 0:
    print(f"\nVerses with multiple Rahman occurrences: {len(multiple_rahman)}")
    display(multiple_rahman[[
        'surah_no', 'ayah_no', 'ayah', 'rahman_frequency', 'rahman_word_locations'
    ]])

print("\n" + "=" * 70)
print("All steps completed successfully!")
print("=" * 70)

FINAL STATISTICS

Dataset Summary:
  Total verses in Quran: 6236
  Verses containing Rahman: 0
  Verses without Rahman: 6236
  Total Rahman occurrences: 0

Rahman Frequency Distribution:

All steps completed successfully!
